In [1]:
import requests
import urllib
import pandas as pd

In [2]:
df = pd.read_csv('./2020/Divvy_Trips_2020_Q1.csv')
df.drop(columns=['ride_id','rideable_type','member_casual'],inplace=True)
df.head()

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng
0,1/1/2020 0:04,1/1/2020 0:17,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246.0,41.9363,-87.6527,41.9561,-87.6688
1,1/1/2020 0:10,1/1/2020 0:10,Daley Center Plaza,81,Daley Center Plaza,81.0,41.8842,-87.6296,41.8842,-87.6296
2,1/1/2020 0:11,1/1/2020 0:15,Daley Center Plaza,81,Dearborn St & Van Buren St,624.0,41.8842,-87.6296,41.8763,-87.6292
3,1/1/2020 0:11,1/1/2020 0:13,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256.0,41.9542,-87.6544,41.9528,-87.6500
4,1/1/2020 0:12,1/1/2020 0:14,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59.0,41.8675,-87.6322,41.8672,-87.6260


In [3]:
df['lat_diff'] = df.end_lat-df.start_lat
df['lon_diff'] = df.end_lng-df.start_lng
df.head()

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,lat_diff,lon_diff
0,1/1/2020 0:04,1/1/2020 0:17,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246.0,41.9363,-87.6527,41.9561,-87.6688,0.0198,-0.0161
1,1/1/2020 0:10,1/1/2020 0:10,Daley Center Plaza,81,Daley Center Plaza,81.0,41.8842,-87.6296,41.8842,-87.6296,0.0000,0.0000
2,1/1/2020 0:11,1/1/2020 0:15,Daley Center Plaza,81,Dearborn St & Van Buren St,624.0,41.8842,-87.6296,41.8763,-87.6292,-0.0079,0.0004
3,1/1/2020 0:11,1/1/2020 0:13,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256.0,41.9542,-87.6544,41.9528,-87.6500,-0.0014,0.0044
4,1/1/2020 0:12,1/1/2020 0:14,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59.0,41.8675,-87.6322,41.8672,-87.6260,-0.0003,0.0062


In [4]:
df=df[df['started_at'] != df['ended_at']]
df=df[df['start_station_name'] != 'HQ QR']
df.shape

(420620, 12)

In [5]:
df.rename(columns={"start_station_id": "from_station_id", "end_station_id": "to_station_id","started_at":"start_time",
                   "ended_at" : "end_time"
                  },inplace=True)
df.head()

,start_time,end_time,start_station_name,from_station_id,end_station_name,to_station_id,start_lat,start_lng,end_lat,end_lng,lat_diff,lon_diff
0,1/1/2020 0:04,1/1/2020 0:17,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246.0,41.9363,-87.6527,41.9561,-87.6688,0.0198,-0.0161
2,1/1/2020 0:11,1/1/2020 0:15,Daley Center Plaza,81,Dearborn St & Van Buren St,624.0,41.8842,-87.6296,41.8763,-87.6292,-0.0079,0.0004
3,1/1/2020 0:11,1/1/2020 0:13,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256.0,41.9542,-87.6544,41.9528,-87.6500,-0.0014,0.0044
4,1/1/2020 0:12,1/1/2020 0:14,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59.0,41.8675,-87.6322,41.8672,-87.6260,-0.0003,0.0062
5,1/1/2020 0:19,1/1/2020 0:26,Clark St & Leland Ave,326,Southport Ave & Irving Park Rd,318.0,41.9671,-87.6674,41.9542,-87.6644,-0.0129,0.0030


In [6]:
paths = df[(df['lat_diff'] > 0.0001) & (df['lon_diff'] > 0.0001)]

In [8]:
import json
with open('./routes.json') as f:
    routes = json.load(f)
keys_len= len(routes.keys())

In [17]:
len(routes.keys())

11269

In [19]:
url='https://api.mapbox.com/directions/v5/mapbox/cycling/'
token='<insert the api key>'

a = df.drop_duplicates(subset=['from_station_id', 'to_station_id'],keep="first")
a = a[(abs(a['lat_diff']) > 0.00001) & (abs(a['lon_diff']) > 0.00001)]

count=0;
for index,rows in a[15500:16000].iterrows():
    from_to=str(int(rows['from_station_id']))+"-"+str(int(rows['to_station_id']))
    try:
        routes[from_to]
    except:
        try:
            routes[str(int(rows['to_station_id']))+"-"+str(int(rows['from_station_id']))]
        except:
            count+=1
# #             print(str(int(rows['from_station_id']))+"-"+str(int(rows['to_station_id'])),str(int(rows['to_station_id']))+"-"+str(int(rows['from_station_id'])))
            lon_lat=str(rows['start_lng'])+","+str(rows['start_lat'])+";"+str(rows['end_lng'])+","+str(rows['end_lat'])
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
            distance = requests.get(url+lon_lat+'?alternatives=true&geometries=geojson&steps=true&'+token,headers= headers)
            dict = distance.json()
            routes[from_to]=dict['routes'][0]['geometry']['coordinates']
print(a.shape,count)

(51440, 12) 313


In [20]:
if  len(routes.keys()) > keys_len:
    with open('./routes.json', 'w', encoding='utf-8') as f:
        json.dump(routes, f, ensure_ascii=False, indent=4)
    print('Writing')
    keys_len= len(routes.keys())
else:
    print('Refresh Data')

Writing
